In [ ]:
import numpy as np
from numpy.core.umath import isnan
import torch
import torch.nn as nn
from models.pointnet2_reg import pointnet2_reg_ptp, reg_loss
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# model init
checkpoint = 'work_dirs/checkpoints/pointnet2_regptp_1000.pth'
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model = pointnet2_reg_ptp(6)
model = model.to(device)
model.load_state_dict(torch.load(checkpoint, map_location=device))
model.eval()


In [ ]:
# read *.stl
def stl_read(shape_file):
    import open3d as o3d
    my_mesh = o3d.io.read_triangle_mesh(shape_file)
    Points = np.asarray(my_mesh.vertices)
    Connectivity = np.asarray(my_mesh.triangles)

    # rescale to 25 m in X-axis
    rate = 25/((Points[:,0]).max()-(Points[:,0]).min())
    Points = Points * rate
    Points[:,0] = Points[:,0] - Points[:,0].mean()
    
    return Points, Connectivity, rate

# more feature of pc
def ex_feature(Points, Connectivity):
    normals = np.zeros(np.shape(Connectivity))
    for i in range(len(Connectivity)):
        temp1 = np.cross(Points[Connectivity[i,2],:] - Points[Connectivity[i,0],:],
            Points[Connectivity[i,1],:] - Points[Connectivity[i,0],:]); # mesh outer direction
        normals[i,:] = temp1/np.linalg.norm(temp1,ord=2)
    normals = -normals
    
    # tri face points
    Points_tri = np.zeros(np.shape(Connectivity))
    for i in range(len(Connectivity)):
        Points_tri[i,:] = np.mean(Points[Connectivity[i,:],:],axis=0)

    # cal tri-area
    tri_point = np.zeros((np.size(Connectivity,0),3,3))
    side_len = np.zeros((np.size(Connectivity,0),3))
    for i in range(3):
        tri_point[:,:,i] = Points[Connectivity[:,i],:]

    for i in range(3):
        tmp = np.array([i,i+1]).astype(int)
        tmp[tmp>=3] = 0
        side_len[:,i] = np.sqrt(np.sum(
            (tri_point[:,:,tmp[0]] - tri_point[:,:,tmp[1]])**2
            ,1))
    side_p = np.sum(side_len,1)/2
    tri_area = np.sqrt(side_p*
        (side_p-side_len[:,0]) *
        (side_p-side_len[:,1]) *
        (side_p-side_len[:,2]))
    tri_area[isnan(tri_area)]=0
    return normals, Points_tri, tri_area

# rotate Y-axis
def rotate_pc(pc, rotation_angle):
    
    cosval = np.cos(np.deg2rad(rotation_angle))
    sinval = np.sin(np.deg2rad(rotation_angle))
    rotation_matrix = np.array([[cosval, 0, sinval],
                                [0, 1, 0],
                                [-sinval, 0, cosval]])
    rotated_pc = np.dot(pc, rotation_matrix)
    return rotated_pc
# pred function
def cp_predict_function(Points_tri, normals, model, sample_iter=5):
    npoints = 1024
    import_shape = np.concatenate((Points_tri,normals),axis=1)
    output = np.zeros((len(import_shape),sample_iter))
    for j in range(sample_iter):
        idx = np.arange(len(import_shape))
        np.random.shuffle(idx) # random index

        shape1 = torch.tensor(import_shape[idx,:])
        shape1 = torch.unsqueeze(shape1, 0)
        
        # division
        div = range(0,shape1.size()[1],npoints)
        xyz = torch.zeros(len(div),npoints,3)
        points = torch.zeros(len(div),npoints,3)

        for i in range(len(div)-1):
            xyz[i,:,:] = shape1[:,div[i]:div[i+1],:3].type(torch.FloatTensor)
            points[i,:,:] = shape1[:,div[i]:div[i+1],3:6].type(torch.FloatTensor)

        # if less than npoints, filling them
        xyz[len(div)-1,:(shape1.size()[1]-div[-1]),:] = shape1[:,div[-1]:,:3].type(torch.FloatTensor)
        points[len(div)-1,:(shape1.size()[1]-div[-1]),:] = shape1[:,div[-1]:,3:6].type(torch.FloatTensor)
        idx_tmp = np.arange(0,div[-1],1,dtype=np.int16)
        np.random.shuffle(idx_tmp)
        idx_tmp = idx_tmp[:(shape1.size()[1]-div[-1])]
        points[len(div)-1,-(shape1.size()[1]-div[-1]):,:] = shape1[:,idx_tmp,3:6].type(torch.FloatTensor)
        xyz[len(div)-1,-(shape1.size()[1]-div[-1]):,:] = shape1[:,idx_tmp,:3].type(torch.FloatTensor)
        
        # prediction
        with torch.no_grad():
            pred = model(xyz.to(device), points.to(device))
            pred = pred.cpu()

        # collect
        pred_ms = np.reshape(np.array(pred),-1)
        pred_ms = pred[:shape1.size()[1]].flatten()
        # pred_ms = np.expand_dims(pred,1)
        

        # reorder
        output[idx,j] = pred_ms[:len(idx)]
        # for i in range(len(idx)):
        #     output[idx[i],j] = pred_ms[i]

        print('iter: '+str(j+1) + '/' +str(sample_iter))

    mu = np.mean(output,axis=1)
    sig = np.std(output,axis=1)
    filter_data = np.zeros(output.shape)
    filter_min = np.tile((mu-sig).reshape(-1,1),5)
    filter_max  = np.tile((mu+sig).reshape(-1,1),5)
    keep = output>=filter_min
    keep = keep & (output<=filter_max)
    filter_num = np.sum(keep,axis=1)
    filter_data[keep] = output[keep]
    filter_data = np.sum(filter_data,axis=1)/filter_num
    filter_data
    pred = filter_data

    return pred


In [ ]:
shape_file = 'STLshape\\spaceplane.stl'

# Tri-reading
if shape_file[-3:] == 'stl' or 'ply':
    Points_init, Connectivity, rate = stl_read(shape_file)

In [ ]:
# AoA rotate
init_aoa = 0 # AOA of the initial STL shape
target_aoa = 15 # target AOA
rotat_aoa = init_aoa - target_aoa 
Points = rotate_pc(Points_init, rotat_aoa)

# Cal normals and area of the tris
normals, Points_tri, tri_area = ex_feature(Points, Connectivity)
Points_tri = np.array(Points_tri)
normals = np.array(normals)
S_ref = 1 # reference area

In [ ]:
if True:
    import plotly.figure_factory as ff

    x = Points[:,0]
    y = Points[:,1]
    z = Points[:,2]

    simplices = Connectivity
    fig = ff.create_trisurf(x=x, y=y, z=z,
                            simplices=simplices,
                            height=600,
                            width=900,
                            title="Import shape")
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=0),
        autosize=False,
        scene_camera_eye=dict(x=-1.8, y=1.8, z=.18),
        scene=dict(aspectmode='data')
    )
    fig.show()

- PointNet++

In [ ]:
cp_predict = lambda Points_tri,normals : cp_predict_function(Points_tri,normals, model)
pred = cp_predict(Points_tri,normals)
coforce_pred = -np.sum(np.repeat(np.reshape(pred*tri_area,(-1,1)),3,axis=1)*normals,axis=0)
coforce_pred = coforce_pred/ S_ref

- Visualize pressure

In [ ]:
if True:
    import plotly.figure_factory as ff

    x = Points[:,0]
    y = Points[:,1]
    z = Points[:,2]

    simplices = Connectivity
    fig = ff.create_trisurf(x=x, y=y, z=z,color_func=pred,
                            colormap="Viridis",
                            simplices=simplices,
                            plot_edges=False,
                            height=600,
                            width=900,
                            title="Pred. pressure")
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=0),
        autosize=False,
        scene_camera_eye=dict(x=-1.8, y=1.8, z=.18),
        scene=dict(aspectmode='data')
    )
    fig.show()

In [ ]:
print('CD: ' + str(coforce_pred[0]) )
print('CL: ' + str(coforce_pred[2]) )